# Creator Metrics

This script processes a CSV from [this Dune Query](https://dune.com/queries/4087193) and derives metrics the following NFT creator metrics:

- num drops
- num unique minters
- num txns
- USD value of txns

In [1]:
from dotenv import load_dotenv
from dune_client.types import QueryParameter
from dune_client.client import DuneClient
from dune_client.query import QueryBase
import json
import os
import pandas as pd

2024-10-02 11:10:02,086 INFO numexpr.utils NumExpr defaulting to 8 threads.


In [2]:
load_dotenv()

DUNE_API_KEY = os.getenv('DUNE_API_KEY')
dune = DuneClient(DUNE_API_KEY)

## Part 1. Parse the creator data

In [3]:
apps = pd.read_csv('data/apps/creator_apps.csv', index_col=0)
apps = apps.drop(columns=['chain']).drop_duplicates()
uuid_to_address_mapper = apps.set_index('uuid')['address'].to_dict()
uuid_list = list(apps['uuid'].unique())
address_list = list(apps['address'].unique())
apps.tail(1)

,uuid,recipient,project_type,category,address
1389,568706f3-8127-448f-b0b1-56c28e4f3ed1,0xf56E55e35d2CCa5A34F5Ba568454974424aEA0F4,Creator,Art NFTs,0xf56e55e35d2cca5a34f5ba568454974424aea0f4


In [4]:
all_apps = pd.read_csv('data/apps/applications_reviewed.csv')
all_apps = all_apps[all_apps['uuid'].isin(uuid_list)]
uuid_to_profile_mapper = all_apps.set_index('uuid')['profile_name'].to_dict()
uuid_to_name_mapper = all_apps.set_index('uuid')['name'].to_dict()
uuid_to_id_mapper = all_apps.set_index('uuid')['id'].to_dict()
all_apps.tail(1)

,uuid,charmverseId,agoraProjectRefUID,id,recipient,time,name,status,profile_name,profile_url,...,chain_id,chain,flag_multiple_projects_same_profile,flag_creator_no_address,flag_app_missing_contract,flag_channel_no_channel,flag_charmverse_in_name,flag_creator_address_conflict,count_flags,has_flag
1613,568706f3-8127-448f-b0b1-56c28e4f3ed1,568706f3-8127-448f-b0b1-56c28e4f3ed1,0x2b24be9c82b7cbd266af7a2a27fbd3213c98bd93c01d...,0xb07fdf1ede14ea0abf845c015d4ad73f336e50acb273...,0xf56E55e35d2CCa5A34F5Ba568454974424aEA0F4,1725398873,面白い事実,pending,music-guy.eth,https://warpcast.com/music-guy.eth,...,NaN,All Superchain,False,False,False,False,False,False,0,0


In [5]:
app_dict = {}
for _,app in apps.iterrows():
    recipient = app['recipient'].lower()
    uuid = app['uuid']
    minting_address = app['address'].lower()
    if recipient not in app_dict:
        app_dict.update({
            recipient: {
                'mintingAddressList': [minting_address],
                'uuidList': [uuid]
            }
        })
    else:
        app_dict[recipient]['mintingAddressList'].append(minting_address)
        app_dict[recipient]['uuidList'].append(uuid)

app_json = []        
for recipient, record in app_dict.items():
    app_json.append({
        'recipientAddress': recipient,
        'mintingAddressList': list(set(record['mintingAddressList'])),
        'uuidList': list(set(record['uuidList'])),
    })
        
with open("data/apps/creator_address_to_uuid.json", "w") as f:
    json.dump(app_json, f, indent=2)

In [6]:
farcaster_df = pd.read_parquet('data/raw_metric_data/farcaster.parquet')
fids = farcaster_df.set_index('address')['fid'].to_dict()

## Part 2. Grab NFT mint data from Dune

In [7]:
creator_addresses_str = ',\n\t\t'.join(address_list)
query_sql = f"""

with nft_creators as (
    select
        nft_contract_address,
        blockchain,
        min_by(tx_from, block_number) as creator_address
    from nft.mints
    where
        block_date between date('2024-06-01') and date('2024-09-01')
        and blockchain in ('base', 'optimism', 'zora')
    group by 1,2
)

select
    nft_creators.creator_address,
    nft_creators.nft_contract_address,
    nft_creators.blockchain,
    count(distinct m.tx_hash) as num_transactions,
    sum(coalesce(m.amount_usd,0)) as amount_usd,
    array_agg(distinct m.buyer) as list_of_buyers
from nft.mints as m
join nft_creators
    on m.nft_contract_address = nft_creators.nft_contract_address
    and m.blockchain = nft_creators.blockchain
where
    m.block_date between date('2024-06-01') and date('2024-09-01')
    and nft_creators.creator_address in (
                {creator_addresses_str}
    )
group by 1,2,3

"""

In [8]:
# query_id = dune.create_query(name='sunny_nft_mints', query_sql=query_sql, is_private=False)
# query = QueryBase(name='sunny_nft_mints', query_id=query_id.base.query_id)
# results_df = dune.run_query_dataframe(query)
# results_df.to_parquet('data/raw_metric_data/dune_raw_nft_mints.parquet')

nft_creators = pd.read_parquet('data/raw_metric_data/dune_raw_nft_mints.parquet')
nft_creators['list_of_buyers'] = nft_creators['list_of_buyers'].apply(lambda x: x.replace("[","").replace("]","").split(" "))
nft_creators['blockchain'] = nft_creators['blockchain'].apply(lambda x: x.title())
nft_creators['list_of_fids'] = nft_creators['list_of_buyers'].apply(
    lambda buyers: [x for x in set([fids.get(x) for x in buyers]) if x is not None]
)
nft_creators.tail(1)

,creator_address,nft_contract_address,blockchain,num_transactions,amount_usd,list_of_buyers,list_of_fids
1150,0xc199d54be70c9837449b8ded44b6c960dde9dfe3,0x2b89241422f0df56fe8837fa5c2018746b33eca7,Base,23,62.691989,"[0x5bfeb4ca066c9458842ac89b6e5cd983bd1a1034, 0...","[14464, 436325, 386213, 7464, 581001, 294090, ..."


# Part 3. Derive the metrics

In [9]:
metrics_by_address = pd.concat([
    nft_creators.groupby('creator_address')['nft_contract_address'].nunique().rename('num_drops'),
    nft_creators.groupby('creator_address')['list_of_buyers'].agg(lambda x: len(set().union(*x))).rename('num_unique_minters'),
    nft_creators.groupby('creator_address')['list_of_fids'].agg(lambda x: len(set().union(*x))).rename('num_farcaster_minters'),
    nft_creators.groupby('creator_address')['num_transactions'].sum().rename('num_transactions'),
    nft_creators.groupby('creator_address')['amount_usd'].sum().rename('usd_value_of_transactions')
], axis=1)
metrics_by_address

,num_drops,num_unique_minters,num_farcaster_minters,num_transactions,usd_value_of_transactions
creator_address,,,,,
0x00409fc839a2ec2e6d12305423d37cd011279c09,3,1,1,3,5.028961
0x0438053b82cb27004bd80d3ca18b8ef6b23f20cf,2,30,25,64,3.055382
0x05a7eb929209f5c1f2f08dd087b64bb1beba99fe,2,15,13,19,0.000000
0x05e0ccdee8e642388da4478facc6ac7c496714f6,1,2,2,3,0.000000
0x07bd4932d08e5956fab224965a58bf1a441134a1,17,47761,6730,50523,16454.105413
...,...,...,...,...,...
0xf75b89857cd1aed40f5b9cf39994775de25ad1f5,1,575,306,610,4444.490493
0xf89afaa93aa0d834e5937f12383faeef41285612,22,10,3,52,48.447741
0xf8c80ebf80ff0c17bdfbc08a91e64a890aae8e3c,7,2894,1032,3094,3655.058326


In [10]:
metrics = []
errors = []
for uuid, app in apps.set_index('uuid').iterrows():
    
    name = uuid_to_name_mapper.get(uuid)
    address = app['address']
    recipient = app['recipient']
    
    if address not in metrics_by_address.index:
        errors.append({
            'uuid': uuid,
            **app,
            'status': 'no_nft_activity_90D'
        })
        continue
    
    creator_metrics = metrics_by_address.loc[address].to_dict()
    
    metrics.append({
        'uuid': uuid,
        **app,
        'status': 'metrics_available',
        **creator_metrics,
    })

In [11]:
df_metrics = pd.DataFrame(metrics)
df_metrics.to_csv('data/clean_metric_data/metrics_creators.csv')

## Part 4. Derive metrics by recipient address

In [12]:
recipient_mapping = (
    all_apps
    .groupby(['recipient', 'profile_name'])
    ['address']
    .unique()
    .apply(list)
    .to_dict()
)

In [13]:
recipient_metrics = []
for (recipient,profile_name),address_list in recipient_mapping.items():

    dff = nft_creators[nft_creators.creator_address.isin(address_list)]
    if len(dff) < 1:
        continue

    uuid_list =  all_apps[(all_apps['recipient'] == recipient) & (all_apps['profile_name'] == profile_name)]['uuid'].unique()
    app_id_list = list(set([uuid_to_id_mapper[x] for x in uuid_list]))
    name_list = list(set([uuid_to_name_mapper[x] for x in uuid_list]))
    
    creator_metrics = {
        'recipient': recipient,
        'artist_profile': profile_name,
    
        'num_drops': dff['nft_contract_address'].nunique(),
        'num_unique_minters': len(set(dff['list_of_buyers'].sum())),
        'num_farcaster_minters': len(set(dff['list_of_fids'].sum())),
        'num_transactions': dff['num_transactions'].sum(),
        'usd_value_of_transactions': dff['amount_usd'].sum(),
        
        'uuid_list': uuid_list,
        'application_id_list': app_id_list,
        'project_names': name_list
    }
    recipient_metrics.append(creator_metrics)

In [14]:
df_recipient_metrics = pd.DataFrame(recipient_metrics).set_index(['recipient', 'artist_profile'])
df_recipient_metrics.to_csv('data/clean_metric_data/metrics_creators_by_recipient.csv')
df_recipient_metrics.sort_values(by='num_transactions')

,,num_drops,num_unique_minters,num_farcaster_minters,num_transactions,usd_value_of_transactions,uuid_list,application_id_list,project_names
recipient,artist_profile,,,,,,,,
0xF15132421A7Cd4a3528d5b938b53c782Fe7fE57D,mayowalawal.eth,1,1,1,1,0.339187,[2612c31b-b3d0-475a-85dc-650fbbfad308],[0x647b118b702796f479900d3ad4275b3fca7d2d8135f...,[Thousand Faces of Bloom]
0x196B5B69bDEf5Aa71d7A18b5208Aa7be23CdE024,bangyadi,1,1,1,1,0.302155,[9b429ca3-10b8-4205-a590-b17028ac678b],[0xc191af54d8eaa24f6001894d09b32d66d32163d5ce7...,[Silhouette]
0x2Ad69BEE4b9858421Fd62f37A9476595e74ad3e6,optimistizu,1,1,1,1,0.000000,[9b25035d-4916-4326-b1cd-ec847ed8da2f],[0xbbaeda9c59c89fbdf705ce2a73a2484096af5b1d38a...,[Optimistizu]
0xc609Cfa28553a2Cd21176b2849F27824090A8aA3,skyncrypto,1,1,1,1,0.000000,[f5e827fd-533d-4099-b4f0-33e36b50eb12],[0xc5e78a8a36ffe814631480bd00584d0c384b80ab1cf...,[Empower with Crypto]
0x6a7415B36133C6C3957e4772B8009068e170C648,ethan666,1,1,0,1,0.000000,[97b4b151-a73c-41f2-88e4-462f1ae4f674],[0x800402f4f35a4b972ff3b32c60d2217e259148abee4...,[Ethan666.eth]
...,...,...,...,...,...,...,...,...,...
0x75C83356987c8d813829D9FBb5DE504b547750A6,goyabean.eth,13,52505,8963,64951,25802.881893,"[68c3c1ab-b463-4bac-9ac9-d082714de866, 577bb64...",[0x327c41e0a5575b80943ef04e740575528d1aa744980...,"[Happy Noiniversary from Based Nouns!, BASED N..."
0x829C0F59FF906fd617F84f6790AF18f440D0C108,mx1000,6,69678,10150,72901,20416.515994,"[dbdf5a7b-9635-4703-ad41-7fb29f96f88d, 84c008f...",[0xeed93adace42d0a55cc0535dc130c0bc4e8348d3052...,"[Chase The Base, Live Countdown NFT for Onchai..."
0x212fC47e707eD36409154ebf8bCfd2aA0E26517B,0xhellia,9,61215,9961,83196,20927.591097,[ccd139de-e7b6-4f9c-8058-b46b9e34bb23],[0x07eef6001515b6c4e36ccd93468779a737a6c2ca77b...,[Cryptopolis]
